## Let's start with downloading necessary libraries

In [1]:
import gym
import numpy as np
from gym import wrappers

from numpy import *
from numpy.random import uniform, normal
from numpy.linalg import norm

from random import shuffle
from collections import deque
from statistics import mean

In [2]:
env = gym.make('CartPole-v1')

## Declare global variables

In [3]:
alpha = 2.0e-1           # the 'learning rate'
maxEpisodes = 1000       # run the agent for 'maxEpisodes'
maxTimeSteps = 500       # maximum number of steps per episode
fixedNorm = 0.5          # output weights are scaled to have norm = 'fixedform'
maxHistory = 5000        # maximum number of recent observations for replay
solvedEpisodes = 100     # cartpole is solved when average reward > 195 for 'solvedEpisodes'
episodeLength = 500      # the target for CartPole-v1

### Observations Transform

In [4]:
inputLength = 4        # length of an observation vector
expansionFactor = 30   # expand observation dimensions by 'expansionFactor'
expandedLength = expansionFactor*inputLength # length of transformed observations

### Feature transform with fixed random weights

In [5]:
V = normal(scale=1.0, size=(expandedLength, inputLength))

### Output weights, randomly initialized

In [6]:
W = uniform(low=-1.0, high=1.0, size=expandedLength)

#### Fix the norm of the output weights to 'fixedNorm'

In [7]:
W *= fixedNorm/norm(W)

# Cart Pole NN implementation

In [8]:
def CartPoleAgent(aplha, W, V):
    #--------------------------------------------
    # Observation history
    H = deque([], maxHistory)
    # episode total reward history
    R = deque([], solvedEpisodes)
    # histories of positive and negative outputs
    PO = deque([0], maxHistory)
    NO = deque([0], maxHistory)
    #--------------------------------------------
    for episode in range(maxEpisodes):
        observation = env.reset()
        H.append(observation)
        totalReward = 0
        for t in range(1, maxTimeSteps + 1):
            env.render()
            #--------------------------------------------
            out = dot(tanh(dot(V, observation)), W)
            if out < 0:
                NO.append(out)
                action = 0
            else:
                PO.append(out)
                action = 1
            #--------------------------------------------
            observation, reward, done, info = env.step(action)
            H.append(observation)
            totalReward += reward
            #--------------------------------------------
            if done:
                R.append(totalReward)
                if t < episodeLength:
                    #--------------------------------------------
                    # Replay shuffled past observation using the
                    # latest weights
                    # Use the means of past outputs as
                    # LMS algorithm target outputs
                    #--------------------------------------------
                    mn = mean(NO)
                    mp = mean(PO)
                    shuffle(H)
                    for obs in H:
                        h = tanh(dot(V, obs))     # transform the observation
                        out = dot(h, W)
                        if out < 0:
                            e = mn - out
                        else:
                            e = mp - out
                        W += aplha * e * h        # Widrow-Hoff LMS update
                        W *= fixedNorm/norm(W)    # keep the weights at fixed norm
                        #---------------------------------------------
                    #--------------------------------------------------
                    avgReward = sum(R)/solvedEpisodes
                    print(f"[{episode:3d}:{totalReward:3.0f}] R:{avgReward:6.2f} mp:{mean(PO):7.3f} mn:{mean(NO):7.3f}  len(H):{len(H):4d}  W:{W[:2]}", flush=True)
                    #---------------------------------------------
                    if avgReward == episodeLength:
                        print("Solved")
                        return
                    #---------------------------------------------
                    break
                #---------------------------------------------
            #---------------------------------------------